In [2]:
import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [4]:
train.head()

,Won_Championship,Previous_SB_Wins,Number_Of_Wins_This_Season,Number_Of_First_Round_Draft_Picks,Team_Value,Playing_Style,Average_Player_Age,Number_Of_Injured_Players,Coach_Experience_Level,ID
0,0,3,13,2,Less_Than_Four_Billion,Balanced,27,five,Intermediate,6056
1,0,2,14,2,Less_Than_Four_Billion,Aggressive_Offense,26,five,Intermediate,9702
2,1,2,13,1,Less_Than_Four_Billion,Aggressive_Defense,27,four,Intermediate,1745
3,0,2,12,2,Above_Four_Billion,Balanced,27,six,Intermediate,4757
4,0,1,15,2,Less_Than_Four_Billion,Balanced,26,five,Intermediate,7242


In [5]:
train['Won_Championship'].value_counts()

1    3255
0    3245
Name: Won_Championship, dtype: int64

In [8]:
pd.crosstab(train['Won_Championship'], train['Coach_Experience_Level'])

Coach_Experience_Level,Advanced,Beginner,Intermediate
Won_Championship,,,
0,76,542,2627
1,596,132,2527


In [9]:
pd.crosstab(train['Team_Value'], train['Coach_Experience_Level'])

Coach_Experience_Level,Advanced,Beginner,Intermediate
Team_Value,,,
Above_Four_Billion,79,69,569
Less_Than_Four_Billion,525,538,4067
Less_Than_Three_Billion,68,67,518


In [10]:
pd.crosstab(train['Team_Value'], train['Won_Championship'])

Won_Championship,0,1
Team_Value,,
Above_Four_Billion,369,348
Less_Than_Four_Billion,2545,2585
Less_Than_Three_Billion,331,322


In [13]:
pd.crosstab(train['Playing_Style'], train['Won_Championship'])

Won_Championship,0,1
Playing_Style,,
Aggressive_Defense,1602,1602
Aggressive_Offense,117,106
Balanced,1444,1464
Relaxed,82,83


In [14]:
train['Number_Of_Wins_This_Season'].value_counts()

14    2269
13    1636
15    1481
12     542
16     421
11      82
17      56
10       7
18       6
Name: Number_Of_Wins_This_Season, dtype: int64

In [15]:
pd.crosstab(train['Number_Of_Wins_This_Season'], train['Won_Championship'])

Won_Championship,0,1
Number_Of_Wins_This_Season,,
10,5,2
11,46,36
12,285,257
13,787,849
14,1115,1154
15,711,770
16,258,163
17,36,20
18,2,4


In [16]:
train['Average_Player_Age'].value_counts()

26    3786
27    2385
25     262
28      66
24       1
Name: Average_Player_Age, dtype: int64

In [17]:
pd.crosstab(train['Average_Player_Age'], train['Won_Championship'])

Won_Championship,0,1
Average_Player_Age,,
24,1,0
25,176,86
26,1842,1944
27,1182,1203
28,44,22


In [11]:
train['Number_Of_Injured_Players'].value_counts()

five     1894
six      1799
four     1069
seven     972
three     343
eight     289
two        78
nine       44
one        10
ten         2
Name: Number_Of_Injured_Players, dtype: int64

In [29]:
test_id = test['ID']

In [30]:
test = test.drop('ID', axis=1)

In [40]:
train = train.drop('ID', axis=1)

In [31]:
categorical_feats = [
    f for f in train.columns if train[f].dtype == 'object'
]

categorical_feats
for f_ in categorical_feats:
    train[f_], _ = pd.factorize(train[f_])
    # Set feature type as categorical
    train[f_] = train[f_].astype('category').cat.codes

In [ ]:
categorical_feats = [
    f for f in df2.columns if df2[f].dtype == 'float64'
]

categorical_feats
for f_ in categorical_feats:
    #df2[f_], _ = pd.factorize(df2[f_])
    # Set feature type as categoricalAXC
    df2[f_] = df2[f_].map(df2[f_].value_counts().to_dict())

In [32]:
categorical_feats = [
    f for f in test.columns if test[f].dtype == 'object'
]

categorical_feats
for f_ in categorical_feats:
    test[f_], _ = pd.factorize(test[f_])
    # Set feature type as categorical
    test[f_] = test[f_].astype('category').cat.codes

In [33]:
train.head()

,Won_Championship,Previous_SB_Wins,Number_Of_Wins_This_Season,Number_Of_First_Round_Draft_Picks,Team_Value,Playing_Style,Average_Player_Age,Number_Of_Injured_Players,Coach_Experience_Level,ID
0,0,3,13,2,0,0,27,0,0,6056
1,0,2,14,2,0,1,26,0,0,9702
2,1,2,13,1,0,2,27,1,0,1745
3,0,2,12,2,1,0,27,2,0,4757
4,0,1,15,2,0,0,26,0,0,7242


In [41]:
X = train.drop('Won_Championship', axis=1)
y = train['Won_Championship']

In [42]:
from catboost import CatBoostClassifier

In [43]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [44]:
# Split the data into 30% validation and 70% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1234)

In [45]:
model = CatBoostClassifier(#reg_lambda=5,#learning_rate=0.1,#200 best so far
                          loss_function='Logloss', eval_metric='Accuracy',
                           #learning_rate=0.03,
                       metric_period=20)#5

In [72]:
from sklearn.metrics import f1_score,confusion_matrix,roc_auc_score, mean_absolute_error, mean_squared_error, f1_score
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit, GroupKFold
fold=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index],
    y_train, y_test = y[train_index], y[test_index]
    m=model#LGBMRegressor(**params)
    #LGBMRegressor(**params)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100)
    preds=m.predict(X_test)
    print("err: ",f1_score(y_test,preds))
    errcb.append(f1_score(y_test,preds))
    p = m.predict(test)
    y_pred_totcb.append(p)

Learning rate set to 0.092775
0:	learn: 0.7111538	test: 0.7111538	test1: 0.6992308	best: 0.6992308 (0)	total: 2.22ms	remaining: 2.22s
20:	learn: 0.7575000	test: 0.7575000	test1: 0.7376923	best: 0.7376923 (20)	total: 51.9ms	remaining: 2.42s
40:	learn: 0.7657692	test: 0.7657692	test1: 0.7430769	best: 0.7446154 (36)	total: 85.7ms	remaining: 2s
60:	learn: 0.7707692	test: 0.7707692	test1: 0.7461538	best: 0.7461538 (59)	total: 121ms	remaining: 1.85s
80:	learn: 0.7750000	test: 0.7750000	test1: 0.7492308	best: 0.7492308 (74)	total: 153ms	remaining: 1.74s
100:	learn: 0.7786538	test: 0.7786538	test1: 0.7523077	best: 0.7523077 (100)	total: 184ms	remaining: 1.64s


120:	learn: 0.7834615	test: 0.7834615	test1: 0.7530769	best: 0.7538462 (104)	total: 224ms	remaining: 1.63s
140:	learn: 0.7863462	test: 0.7863462	test1: 0.7561538	best: 0.7561538 (134)	total: 261ms	remaining: 1.59s
160:	learn: 0.7878846	test: 0.7878846	test1: 0.7607692	best: 0.7607692 (159)	total: 293ms	remaining: 1.52s
180:	learn: 0.7909615	test: 0.7909615	test1: 0.7600000	best: 0.7623077 (163)	total: 327ms	remaining: 1.48s
200:	learn: 0.7919231	test: 0.7919231	test1: 0.7600000	best: 0.7623077 (163)	total: 360ms	remaining: 1.43s
220:	learn: 0.7951923	test: 0.7951923	test1: 0.7615385	best: 0.7623077 (163)	total: 396ms	remaining: 1.4s
240:	learn: 0.7973077	test: 0.7973077	test1: 0.7607692	best: 0.7623077 (163)	total: 431ms	remaining: 1.36s
260:	learn: 0.7996154	test: 0.7996154	test1: 0.7615385	best: 0.7623077 (163)	total: 465ms	remaining: 1.32s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7623076923
bestIteration = 163

Shrink model to first 164 iterations.
err:  

60:	learn: 0.7576923	test: 0.7576923	test1: 0.7561538	best: 0.7584615 (32)	total: 112ms	remaining: 1.73s
80:	learn: 0.7638462	test: 0.7638462	test1: 0.7630769	best: 0.7646154 (75)	total: 145ms	remaining: 1.65s
100:	learn: 0.7717308	test: 0.7717308	test1: 0.7707692	best: 0.7707692 (100)	total: 179ms	remaining: 1.59s
120:	learn: 0.7784615	test: 0.7784615	test1: 0.7730769	best: 0.7730769 (120)	total: 213ms	remaining: 1.54s
140:	learn: 0.7807692	test: 0.7807692	test1: 0.7761538	best: 0.7776923 (136)	total: 246ms	remaining: 1.5s
160:	learn: 0.7830769	test: 0.7830769	test1: 0.7769231	best: 0.7776923 (136)	total: 279ms	remaining: 1.45s
180:	learn: 0.7834615	test: 0.7834615	test1: 0.7776923	best: 0.7776923 (136)	total: 312ms	remaining: 1.41s
200:	learn: 0.7850000	test: 0.7850000	test1: 0.7776923	best: 0.7776923 (136)	total: 347ms	remaining: 1.38s
220:	learn: 0.7878846	test: 0.7878846	test1: 0.7746154	best: 0.7776923 (136)	total: 385ms	remaining: 1.36s
Stopped by overfitting detector  (100 iter

80:	learn: 0.7698077	test: 0.7698077	test1: 0.7438462	best: 0.7438462 (78)	total: 139ms	remaining: 1.57s
100:	learn: 0.7755769	test: 0.7755769	test1: 0.7469231	best: 0.7469231 (97)	total: 173ms	remaining: 1.54s
120:	learn: 0.7840385	test: 0.7840385	test1: 0.7476923	best: 0.7507692 (106)	total: 209ms	remaining: 1.52s
140:	learn: 0.7859615	test: 0.7859615	test1: 0.7507692	best: 0.7523077 (136)	total: 248ms	remaining: 1.51s
160:	learn: 0.7867308	test: 0.7867308	test1: 0.7515385	best: 0.7523077 (136)	total: 283ms	remaining: 1.48s
180:	learn: 0.7900000	test: 0.7900000	test1: 0.7492308	best: 0.7523077 (136)	total: 320ms	remaining: 1.45s
200:	learn: 0.7923077	test: 0.7923077	test1: 0.7523077	best: 0.7538462 (197)	total: 365ms	remaining: 1.45s
220:	learn: 0.7951923	test: 0.7951923	test1: 0.7546154	best: 0.7569231 (209)	total: 407ms	remaining: 1.43s
240:	learn: 0.7955769	test: 0.7955769	test1: 0.7592308	best: 0.7592308 (238)	total: 442ms	remaining: 1.39s
260:	learn: 0.7973077	test: 0.7973077	te

80:	learn: 0.7644231	test: 0.7644231	test1: 0.7523077	best: 0.7523077 (79)	total: 151ms	remaining: 1.71s
100:	learn: 0.7700000	test: 0.7700000	test1: 0.7553846	best: 0.7576923 (83)	total: 186ms	remaining: 1.65s
120:	learn: 0.7753846	test: 0.7753846	test1: 0.7600000	best: 0.7600000 (112)	total: 220ms	remaining: 1.59s
140:	learn: 0.7794231	test: 0.7794231	test1: 0.7646154	best: 0.7646154 (134)	total: 252ms	remaining: 1.54s
160:	learn: 0.7805769	test: 0.7805769	test1: 0.7623077	best: 0.7646154 (134)	total: 285ms	remaining: 1.49s
180:	learn: 0.7851923	test: 0.7851923	test1: 0.7669231	best: 0.7669231 (171)	total: 318ms	remaining: 1.44s
200:	learn: 0.7882692	test: 0.7882692	test1: 0.7661538	best: 0.7669231 (171)	total: 352ms	remaining: 1.4s
220:	learn: 0.7898077	test: 0.7898077	test1: 0.7669231	best: 0.7669231 (171)	total: 388ms	remaining: 1.37s
240:	learn: 0.7921154	test: 0.7921154	test1: 0.7653846	best: 0.7669231 (171)	total: 426ms	remaining: 1.34s
260:	learn: 0.7944231	test: 0.7944231	tes

60:	learn: 0.7640385	test: 0.7640385	test1: 0.7546154	best: 0.7576923 (49)	total: 107ms	remaining: 1.65s
80:	learn: 0.7686538	test: 0.7686538	test1: 0.7500000	best: 0.7576923 (49)	total: 146ms	remaining: 1.65s
100:	learn: 0.7721154	test: 0.7721154	test1: 0.7523077	best: 0.7576923 (49)	total: 181ms	remaining: 1.61s
120:	learn: 0.7776923	test: 0.7776923	test1: 0.7538462	best: 0.7576923 (49)	total: 217ms	remaining: 1.58s
140:	learn: 0.7830769	test: 0.7830769	test1: 0.7523077	best: 0.7576923 (49)	total: 251ms	remaining: 1.53s
160:	learn: 0.7867308	test: 0.7867308	test1: 0.7530769	best: 0.7584615 (145)	total: 285ms	remaining: 1.48s
180:	learn: 0.7878846	test: 0.7878846	test1: 0.7600000	best: 0.7600000 (180)	total: 321ms	remaining: 1.45s
200:	learn: 0.7928846	test: 0.7928846	test1: 0.7607692	best: 0.7638462 (183)	total: 353ms	remaining: 1.4s
220:	learn: 0.7950000	test: 0.7950000	test1: 0.7607692	best: 0.7638462 (183)	total: 385ms	remaining: 1.36s
240:	learn: 0.7955769	test: 0.7955769	test1: 

In [73]:
np.mean(errcb)

0.7660267024480636

In [77]:
from sklearn.ensemble import RandomForestClassifier

In [79]:
rf = RandomForestClassifier()

In [80]:
rf.fit(X,y)

C:\Users\nraheem\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Learning rate set to 0.094975
0:	learn: 0.7090769	test: 0.7090769	best: 0.7090769 (0)	total: 2.47ms	remaining: 2.46s
20:	learn: 0.7532308	test: 0.7532308	best: 0.7532308 (20)	total: 42.1ms	remaining: 1.96s
40:	learn: 0.7606154	test: 0.7606154	best: 0.7618462 (38)	total: 83.5ms	remaining: 1.95s
60:	learn: 0.7661538	test: 0.7661538	best: 0.7661538 (60)	total: 124ms	remaining: 1.92s
80:	learn: 0.7712308	test: 0.7712308	best: 0.7713846 (79)	total: 163ms	remaining: 1.85s


100:	learn: 0.7730769	test: 0.7730769	best: 0.7730769 (97)	total: 206ms	remaining: 1.83s
120:	learn: 0.7783077	test: 0.7783077	best: 0.7783077 (114)	total: 245ms	remaining: 1.78s
140:	learn: 0.7801538	test: 0.7801538	best: 0.7804615 (138)	total: 284ms	remaining: 1.73s
160:	learn: 0.7835385	test: 0.7835385	best: 0.7835385 (160)	total: 324ms	remaining: 1.69s
180:	learn: 0.7861538	test: 0.7861538	best: 0.7861538 (179)	total: 362ms	remaining: 1.64s
200:	learn: 0.7873846	test: 0.7873846	best: 0.7883077 (199)	total: 402ms	remaining: 1.6s
220:	learn: 0.7901538	test: 0.7901538	best: 0.7901538 (220)	total: 441ms	remaining: 1.55s
240:	learn: 0.7907692	test: 0.7907692	best: 0.7909231 (236)	total: 481ms	remaining: 1.51s
260:	learn: 0.7915385	test: 0.7915385	best: 0.7915385 (257)	total: 523ms	remaining: 1.48s
280:	learn: 0.7936923	test: 0.7936923	best: 0.7936923 (279)	total: 563ms	remaining: 1.44s
300:	learn: 0.7950769	test: 0.7950769	best: 0.7952308 (294)	total: 604ms	remaining: 1.4s
320:	learn: 0

In [81]:
preds = rf.predict(test)

In [82]:
pd.DataFrame({'ID':test_id, 'Won_Championship':preds}).to_csv('win4.csv', index=False)

In [65]:
preds = np.mean(y_pred_totcb, 0)

In [66]:
pred = pd.DataFrame(preds, columns=['Won_Championship'])

In [67]:
pred['Won_Championship'].value_counts()

0.0    1612
1.0    1119
0.2     247
0.8     223
0.4     150
0.6     149
Name: Won_Championship, dtype: int64

In [68]:
def thresh(x):
    if x <= 0.2:
        return 0
    else:
        return 1

In [69]:
pred['Won_Championship'] = pred['Won_Championship'].apply(thresh)

In [70]:
pred['Won_Championship'].value_counts()

0    1859
1    1641
Name: Won_Championship, dtype: int64

In [71]:
pd.DataFrame({'ID':test_id, 'Won_Championship':pred['Won_Championship']}).to_csv('win2.csv', index=False)